# SPARK Homework

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import pyspark
import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 

<font color='red'>Deadline on 12th april 2022 18:00
</font>

This homework can be realised in pairs (maximum 2 people maximum) or alone.

Please, give all author names :



Rename the notebook including all author names.
 ex DEVOIR_SPARK_SUJET_Lastname1_Lastname2.ipynb


![image.png](attachment:image.png)

## Prerequisite

Start your Spark cluster :

Check all the following containers are up :

smaster, sworker1, sworker2 


### Data download

This practice is based on a dataset from data.gouv.fr.

You will need an internet acces in order to download the dataset. The following cell will download data into `/dataspark` :

In [ ]:
!wget https://files.data.gouv.fr/geo-dvf/latest/csv/2021/full.csv.gz -O 2021-mutations-immobilieres.csv.gz

--2022-04-12 08:33:20--  https://files.data.gouv.fr/geo-dvf/latest/csv/2021/full.csv.gz
Resolving files.data.gouv.fr (files.data.gouv.fr)... 51.38.54.240
Connecting to files.data.gouv.fr (files.data.gouv.fr)|51.38.54.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34657631 (33M) [application/octet-stream]
Saving to: ‘2021-mutations-immobilieres.csv.gz’

2021-mutations-immo 100%[===================>]  33.05M  19.8MB/s    in 1.7s    

2022-04-12 08:33:22 (19.8 MB/s) - ‘2021-mutations-immobilieres.csv.gz’ saved [34657631/34657631]



The dataset represents all France real estate transfer in 2021.

Here is the description of each column : 

```
id_mutation : Identifiant de mutation (non stable, sert à grouper les lignes)
date_mutation : Date de la mutation au format ISO-8601 (YYYY-MM-DD)
numero_disposition : Numéro de disposition
nature_mutation : Nature de la mutation
valeur_fonciere : Valeur foncière (séparateur décimal = point)
adresse_numero : Numéro de l'adresse
adresse_suffixe : Suffixe du numéro de l'adresse (B, T, Q)
adresse_code_voie : Code FANTOIR de la voie (4 caractères)
adresse_nom_voie : Nom de la voie de l'adresse
code_postal : Code postal (5 caractères)
code_commune : Code commune INSEE (5 caractères)
nom_commune : Nom de la commune (accentué)
ancien_code_commune : Ancien code commune INSEE (si différent lors de la mutation)
ancien_nom_commune : Ancien nom de la commune (si différent lors de la mutation)
code_departement : Code département INSEE (2 ou 3 caractères)
id_parcelle : Identifiant de parcelle (14 caractères)
ancien_id_parcelle : Ancien identifiant de parcelle (si différent lors de la mutation)
numero_volume : Numéro de volume
lot_1_numero : Numéro du lot 1
lot_1_surface_carrez : Surface Carrez du lot 1
lot_2_numero : Numéro du lot 2
lot_2_surface_carrez : Surface Carrez du lot 2
lot_3_numero : Numéro du lot 3
lot_3_surface_carrez : Surface Carrez du lot 3
lot_4_numero : Numéro du lot 4
lot_4_surface_carrez : Surface Carrez du lot 4
lot_5_numero : Numéro du lot 5
lot_5_surface_carrez : Surface Carrez du lot 5
nombre_lots : Nombre de lots
code_type_local : Code de type de local
type_local : Libellé du type de local
surface_reelle_bati : Surface réelle du bâti
nombre_pieces_principales : Nombre de pièces principales
code_nature_culture : Code de nature de culture
nature_culture : Libellé de nature de culture
code_nature_culture_speciale : Code de nature de culture spéciale
nature_culture_speciale : Libellé de nature de culture spéciale
surface_terrain : Surface du terrain
longitude : Longitude du centre de la parcelle concernée (WGS-84)
latitude : Latitude du centre de la parcelle concernée (WGS-84)    
```

You are now ready to analyze data.

Note that the following documentation may be a great help to assist you in this homework.

https://sparkbyexamples.com/pyspark-tutorial/


## 1 - Dataframe

Open a connection to your Spark cluster :

In [ ]:
# # N'oubliez pas de fermer la connexion à la fin du TP
# # spark.stop()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("spark://smaster:7077").appName("TPDF02").getOrCreate()

Check your connection :

In [ ]:
spark

<font color='red'>Q1 - Open and infer the dataset into the mutationsDF dataframe ? (1pt)
    
Note that the dataset schema should be automatically included into the dataframe.  
The dataset is compressed under the format .gz, don't worry about that Spark is able to read like a simple a csv file.
</font>

In [ ]:
df = spark.read.csv('2021-mutations-immobilieres.csv.gz', inferSchema=True,header=True)

<font color='red'>Q2 - Display the data schema of mutationsDF dataframe ?  (1pt)
</font>

In [ ]:
df.printSchema()

root
 |-- id_mutation: string (nullable = true)
 |-- date_mutation: string (nullable = true)
 |-- numero_disposition: integer (nullable = true)
 |-- nature_mutation: string (nullable = true)
 |-- valeur_fonciere: double (nullable = true)
 |-- adresse_numero: integer (nullable = true)
 |-- adresse_suffixe: string (nullable = true)
 |-- adresse_nom_voie: string (nullable = true)
 |-- adresse_code_voie: string (nullable = true)
 |-- code_postal: integer (nullable = true)
 |-- code_commune: string (nullable = true)
 |-- nom_commune: string (nullable = true)
 |-- code_departement: string (nullable = true)
 |-- ancien_code_commune: string (nullable = true)
 |-- ancien_nom_commune: string (nullable = true)
 |-- id_parcelle: string (nullable = true)
 |-- ancien_id_parcelle: string (nullable = true)
 |-- numero_volume: string (nullable = true)
 |-- lot1_numero: string (nullable = true)
 |-- lot1_surface_carrez: double (nullable = true)
 |-- lot2_numero: string (nullable = true)
 |-- lot2_surfac

<font color='red'>Q3 - Display one row of your mutationsDF dataframe : (1pt)
    </font>

In [ ]:
df_DateMutation = df.select('date_mutation').show()

+-------------+
|date_mutation|
+-------------+
|   2021-01-05|
|   2021-01-05|
|   2021-01-06|
|   2021-01-04|
|   2021-01-06|
|   2021-01-06|
|   2021-01-04|
|   2021-01-04|
|   2021-01-04|
|   2021-01-11|
|   2021-01-07|
|   2021-01-07|
|   2021-01-07|
|   2021-01-07|
|   2021-01-04|
|   2021-01-08|
|   2021-01-07|
|   2021-01-11|
|   2021-01-08|
|   2021-01-08|
+-------------+
only showing top 20 rows



<font color='red'>Q4 - Create from mutationsDF dataframe a new Dataframe CleanMutationDF which matches the following constraints : (2pt)
* type_local is not null
* type_local is 'Maison' or 'Appartement'
* only the following attributes should be selected : id_mutation , nature_mutation, type_local , date_mutation , valeur_fonciere


</font>

In [ ]:
CleanMutationDF = df.na.drop(subset=['type_local']) # drop null elements
liste = ['Maison', 'Appartement'] 
CleanMutationDF = CleanMutationDF.filter(df.type_local.isin(liste))
selection = ['id_mutation', 'nature_mutation', 'type_local', 'date_mutation', 'valeur_fonciere']
CleanMutationDF = CleanMutationDF.select(selection)
CleanMutationDF.show()

+-----------+---------------+-----------+-------------+---------------+
|id_mutation|nature_mutation| type_local|date_mutation|valeur_fonciere|
+-----------+---------------+-----------+-------------+---------------+
|     2021-1|          Vente|     Maison|   2021-01-05|       185000.0|
|     2021-3|          Vente|     Maison|   2021-01-04|       204332.0|
|     2021-4|          Vente|     Maison|   2021-01-06|       320000.0|
|     2021-6|          Vente|Appartement|   2021-01-04|       176000.0|
|     2021-9|          Vente|     Maison|   2021-01-04|       226700.0|
|    2021-10|          Vente|     Maison|   2021-01-08|       185000.0|
|    2021-11|          Vente|     Maison|   2021-01-07|       114500.0|
|    2021-13|          Vente|     Maison|   2021-01-08|       145000.0|
|    2021-14|          Vente|     Maison|   2021-01-07|       169600.0|
|    2021-15|          Vente|Appartement|   2021-01-08|       129150.0|
|    2021-16|          Vente|     Maison|   2021-01-18|       17

<font color='red'>Q5 - Save your CleanMutationDF dataframe into a parquet file ( /dataspark/mutations-immobilieres.parquet ) ? (1pt)
</font>

In [ ]:
CleanMutationDF.write.parquet("mutations-immobilieres.parquet")

If you need to remove the parquet file you can run the following cell :

In [ ]:
!rm -rf mutations-immobilieres.parquet

<font color='red'>Q6 - Load  the parquet file ('/dataspark/mutations-immobilieres.parquet') into a mutationsPDF dataframe  ? (1pt)</font> 

In [ ]:
mutationsPDF = spark.read.parquet('mutations-immobilieres.parquet')

<font color='red'>Q7 - How many rows do you have in mutationsPDF ? (1pt)</font>

In [ ]:
mutationsPDF.describe().show()
print(f'There is 429006 rows in mutationsPDF')

+-------+-----------+--------------------+-----------+-------------+------------------+
|summary|id_mutation|     nature_mutation| type_local|date_mutation|   valeur_fonciere|
+-------+-----------+--------------------+-----------+-------------+------------------+
|  count|     429006|              429006|     429006|       429006|            427738|
|   mean|       null|                null|       null|         null|370445.28003852925|
| stddev|       null|                null|       null|         null| 1329070.422905814|
|    min|     2021-1|        Adjudication|Appartement|   2021-01-01|               0.5|
|    max| 2021-99998|Vente terrain à b...|     Maison|   2021-06-30|      1.37204368E8|
+-------+-----------+--------------------+-----------+-------------+------------------+

There is 429006 rows in mutationsPDF




<div class="alert alert-block alert-info">
 Please note that there may be several rows for the same transaction. All the rows part of a single transaction have the same identifier (i.e. the same value) in the id_mutation column. For instance, there are two rows with the value 2021-887 in the id_mutation column.
</div>

<font color='red'>Q8 - Select all rows concerning the id_mutation 2021-15481 into the singleTrDF dataframe (1pt) :
</font>


In [ ]:
same_id = '2021-15481'
singleTrDF = mutationsPDF.filter(mutationsPDF.id_mutation == same_id)
singleTrDF.show()


+-----------+---------------+----------+-------------+---------------+
|id_mutation|nature_mutation|type_local|date_mutation|valeur_fonciere|
+-----------+---------------+----------+-------------+---------------+
| 2021-15481|          Vente|    Maison|   2021-01-13|       900000.0|
| 2021-15481|          Vente|    Maison|   2021-01-13|       900000.0|
| 2021-15481|          Vente|    Maison|   2021-01-13|       900000.0|
+-----------+---------------+----------+-------------+---------------+





<font color='red'>Q9 - The singleTrDF contains 3 lines for the same transaction. How could you filtered out duplicated rows ? (1pt) 
</font>

In [ ]:
singleTrDF = singleTrDF.dropDuplicates(['id_mutation'])
singleTrDF.show()

+-----------+---------------+----------+-------------+---------------+
|id_mutation|nature_mutation|type_local|date_mutation|valeur_fonciere|
+-----------+---------------+----------+-------------+---------------+
| 2021-15481|          Vente|    Maison|   2021-01-13|       900000.0|
+-----------+---------------+----------+-------------+---------------+



<font color='red'>Q10 -  From mutationsPDF dataframe, create a new dataframe mutationDistinctPDF matching the following constraints : (1pt)
* no duplicated rows
* selecting only nature_mutation = 'Vente'
</font>

In [ ]:
mutationsPDF.show()

+-----------+---------------+-----------+-------------+---------------+
|id_mutation|nature_mutation| type_local|date_mutation|valeur_fonciere|
+-----------+---------------+-----------+-------------+---------------+
|     2021-1|          Vente|     Maison|   2021-01-05|       185000.0|
|     2021-3|          Vente|     Maison|   2021-01-04|       204332.0|
|     2021-4|          Vente|     Maison|   2021-01-06|       320000.0|
|     2021-6|          Vente|Appartement|   2021-01-04|       176000.0|
|     2021-9|          Vente|     Maison|   2021-01-04|       226700.0|
|    2021-10|          Vente|     Maison|   2021-01-08|       185000.0|
|    2021-11|          Vente|     Maison|   2021-01-07|       114500.0|
|    2021-13|          Vente|     Maison|   2021-01-08|       145000.0|
|    2021-14|          Vente|     Maison|   2021-01-07|       169600.0|
|    2021-15|          Vente|Appartement|   2021-01-08|       129150.0|
|    2021-16|          Vente|     Maison|   2021-01-18|       17

In [ ]:
mutationsPDF = mutationsPDF.filter(mutationsPDF.nature_mutation == 'Vente')
mutationsDistinctPDF = mutationsPDF.dropDuplicates(['id_mutation'])

In [ ]:
mutationsDistinctPDF.show()

+-----------+---------------+-----------+-------------+---------------+
|id_mutation|nature_mutation| type_local|date_mutation|valeur_fonciere|
+-----------+---------------+-----------+-------------+---------------+
|    2021-10|          Vente|     Maison|   2021-01-08|       185000.0|
|   2021-100|          Vente|     Maison|   2021-01-21|        86000.0|
|  2021-1000|          Vente|     Maison|   2021-04-15|       375000.0|
| 2021-10000|          Vente|     Maison|   2021-01-29|       160000.0|
|2021-100004|          Vente|     Maison|   2021-05-07|       345000.0|
|2021-100005|          Vente|     Maison|   2021-04-29|       115000.0|
|2021-100006|          Vente|     Maison|   2021-05-07|       297500.0|
|2021-100008|          Vente|Appartement|   2021-04-22|        76500.0|
|2021-100009|          Vente|     Maison|   2021-04-21|       171100.0|
| 2021-10001|          Vente|     Maison|   2021-02-02|       130000.0|
|2021-100011|          Vente|Appartement|   2021-05-10|       15

In [ ]:
mutationsDistinctPDF.describe().show()

+-------+-----------+---------------+-----------+-------------+------------------+
|summary|id_mutation|nature_mutation| type_local|date_mutation|   valeur_fonciere|
+-------+-----------+---------------+-----------+-------------+------------------+
|  count|     340081|         340081|     340081|       340081|            339840|
|   mean|       null|           null|       null|         null|248178.61635507882|
| stddev|       null|           null|       null|         null| 434001.7678859122|
|    min|     2021-1|          Vente|Appartement|   2021-01-01|               0.5|
|    max| 2021-99996|          Vente|     Maison|   2021-06-30|       7.2277368E7|
+-------+-----------+---------------+-----------+-------------+------------------+



<font color='red'>Q11 -  From mutationDistinctPDF dataframe, compare the sales amount (valeur_fonciere) between 'Maison' (House) and 'Appartment' by month (3 pts)
</font>


<div class="alert alert-block alert-info">
Note that you can use the month function included into the pyspark.sql.functions to extract the month value from a date.
</div>


In [ ]:
from pyspark.sql.functions import month

new_mutationDistinctPDF = mutationsDistinctPDF.withColumn('Month', month(mutationsDistinctPDF['date_mutation']))
# new_mutationDistinctPDF.show()
new_mutationDistinctPDF = new_mutationDistinctPDF.select("valeur_fonciere","Month",'type_local')
# new_mutationDistinctPDF.show()
new_mutationDistinctPDF = new_mutationDistinctPDF.orderBy('type_local','month').groupBy('type_local','Month').sum()
new_mutationDistinctPDF.show()

+-----------+-----+--------------------+----------+
| type_local|Month|sum(valeur_fonciere)|sum(Month)|
+-----------+-----+--------------------+----------+
|Appartement|    1|      6.0056700509E9|     24735|
|Appartement|    2| 5.515345101689998E9|     46358|
|Appartement|    3| 6.667854342100002E9|     80007|
|Appartement|    4|     6.07096386606E9|     96984|
|Appartement|    5| 5.714139175840002E9|    111555|
|Appartement|    6| 6.648643771089997E9|    145086|
|     Maison|    1| 8.260093426790003E9|     35034|
|     Maison|    2| 7.613533315049997E9|     64778|
|     Maison|    3| 8.591795203330002E9|    106260|
|     Maison|    4| 8.077311011129997E9|    131232|
|     Maison|    5| 7.103062856420001E9|    141905|
|     Maison|    6| 8.072608861709998E9|    184368|
+-----------+-----+--------------------+----------+



<font color='red'>Q12 - Determine the month where sales amount is highest for 'Maison' and 'Appartement' ? (1pt)
</font>

In [ ]:
import pyspark.sql.functions as f 

In [ ]:
new_mutationDistinctPDF.groupBy('type_local').agg(f.max(new_mutationDistinctPDF['sum(valeur_fonciere)']).alias('Highest sales amount')).show()

+-----------+--------------------+
| type_local|Highest sales amount|
+-----------+--------------------+
|Appartement| 6.667854342100002E9|
|     Maison| 8.591795203330002E9|
+-----------+--------------------+



In [ ]:
new_mutationDistinctPDF.filter(new_mutationDistinctPDF['sum(valeur_fonciere)'] == 6.667854342100002E9).show()

+-----------+-----+--------------------+----------+
| type_local|Month|sum(valeur_fonciere)|sum(Month)|
+-----------+-----+--------------------+----------+
|Appartement|    3| 6.667854342100002E9|     80007|
+-----------+-----+--------------------+----------+



In [ ]:
new_mutationDistinctPDF.filter(new_mutationDistinctPDF['sum(valeur_fonciere)'] == 8.591795203330002E9).show()

+----------+-----+--------------------+----------+
|type_local|Month|sum(valeur_fonciere)|sum(Month)|
+----------+-----+--------------------+----------+
|    Maison|    3| 8.591795203330002E9|    106260|
+----------+-----+--------------------+----------+



# 3 - SQL

In this exercise, you will handle the dataset using the SQL language.


<font color='red'>Q13 - From the mutationDistinctPDF dataframe, create a view mutationSalesV ? (1pt) </font>

In [ ]:
mutationsDistinctPDF.createOrReplaceTempView('mutationSalesV')
spark.sql('desc mutationSalesV').show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|    id_mutation|   string|   null|
|nature_mutation|   string|   null|
|     type_local|   string|   null|
|  date_mutation|   string|   null|
|valeur_fonciere|   double|   null|
+---------------+---------+-------+



<font color='red'>Q14 -  From the mutationSalesV view, compare the sales amount (valeur_fonciere) between 'Maison' (House) and 'Appartment' by month using SQL ? (4 pts)

</font>

In [ ]:
sql_sales = spark.sql("SELECT month(date_mutation), type_local, SUM(valeur_fonciere) FROM mutationSalesV GROUP BY month(date_mutation), type_local ORDER BY type_local, month(date_mutation)")
sql_sales.show()

+--------------------+-----------+--------------------+
|month(date_mutation)| type_local|sum(valeur_fonciere)|
+--------------------+-----------+--------------------+
|                   1|Appartement|      6.0056700509E9|
|                   2|Appartement| 5.515345101689999E9|
|                   3|Appartement| 6.667854342100002E9|
|                   4|Appartement| 6.070963866059999E9|
|                   5|Appartement|     5.71413917584E9|
|                   6|Appartement|     6.64864377109E9|
|                   1|     Maison| 8.260093426789999E9|
|                   2|     Maison| 7.613533315049999E9|
|                   3|     Maison| 8.591795203329998E9|
|                   4|     Maison| 8.077311011130001E9|
|                   5|     Maison|     7.10306285642E9|
|                   6|     Maison| 8.072608861709997E9|
+--------------------+-----------+--------------------+



Close your Spark connection.

In [ ]:
spark.stop()